In [1]:
from pyspark.sql import SparkSession
from delta import *
from minio import Minio
import pyspark.sql.functions as F
import os
import pandas as pd

In [2]:
#CONFIGURAÇÕES SPARK
spark = (
    SparkSession.builder.master("local[*]").appName("csvToParquet")
    .config("spark.executor.memory", "16g")
    #HADOOP
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    #DELTA
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.jars", "jars/aws-java-sdk-bundle-1.11.1026.jar, \
            jars/hadoop-aws-3.2.0.jar")
    .getOrCreate()
)


24/10/13 04:36:02 WARN Utils: Your hostname, DESKTOP-CV9TLA8 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/10/13 04:36:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/hacs/.cache/pypoetry/virtualenvs/pyspark-delta-gPM1PAZH-py3.10/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hacs/.ivy2/cache
The jars for the packages stored in: /home/hacs/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-79a79415-e581-4d06-9fda-236e75fbe7cc;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 186ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0

In [3]:
#CONFIGURAÇÕES CLIENT DO MINIO
client=Minio("localhost:9000",
             access_key="minioadmin",
             secret_key="minioadmin",
             secure=False
            )


In [4]:
bucket_name = 'bronze'
objects=client.list_objects(bucket_name, prefix='CAPES/', recursive=True)
csvs = []
for object in objects:
    csvs.append(object.object_name)
    
name_csvs = []
for csv in csvs:
    name_csvs.append(csv.split('/'))

In [5]:
count = 0
for csv in csvs:
    print(f'{count} - {csv}')
    count+=1

0 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022 Bibliográfica Subtipo Livro.csv
1 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Artigo em Jornal ou Revista.csv
2 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Outro.csv
3 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Aplicativo.csv
4 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Produto.csv
5 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnic

In [6]:
def unionDf(dfstart, dfend):
    df_one = spark.read.format('csv').option('header', 'true').option('sep', ';')\
        .load(f's3a://{bucket_name}/{csvs[dfstart]}')
    for i in range(dfstart+1, dfend+1):
        df_two = spark.read.format('csv').option('header', 'true').option('sep', ';')\
        .load(f's3a://{bucket_name}/{csvs[dfend]}')
        df_one = df_one.unionByName(df_two, allowMissingColumns=True)
    return df_one


In [7]:
def clearOnlyParquets(folder):
    for root, dirs, files in os.walk(folder):
        for file in files:
            if not file.endswith(".parquet"):
                os.remove(os.path.join(root, file))

In [8]:
def toParquet(df, nameDf):
    df.write.mode('overwrite').parquet(nameDf)
    clearOnlyParquets(nameDf)

In [40]:
#PROJETOS
df = unionDf(49,50)
print(detalhes_projetos.columns)

['CD_PROGRAMA_IES', 'SG_ENTIDADE_ENSINO', 'NM_ENTIDADE_ENSINO', 'NM_LINHA_PESQUISA', 'DH_INICIO_LINHA', 'ID_PROJETO', 'DS_PROJETO', 'NM_NATUREZA_PROJETO', 'DH_INICIO', 'DT_SITUACAO_ATUAL', 'NM_TIPO_SITUACAO_ATUAL']


In [41]:
projetos = df.select('ID_PROJETO', 'NM_PROJETO')
detalhes_projetos = df.select('CD_PROGRAMA_IES', 'SG_ENTIDADE_ENSINO', 'NM_ENTIDADE_ENSINO',
                                            'NM_LINHA_PESQUISA', 'DH_INICIO_LINHA', 'ID_PROJETO', 'DS_PROJETO', 
                                            'NM_NATUREZA_PROJETO', 'DH_INICIO', 'DT_SITUACAO_ATUAL', 'NM_TIPO_SITUACAO_ATUAL')
projetos = projetos.dropDuplicates(['ID_PROJETO'])
detalhes_projetos = detalhes_projetos.dropDuplicates()

In [42]:
toParquet(projetos, 'capes/capes projetos')
toParquet(detalhes_projetos, 'capes/capes projetos_detalhes')

In [43]:
#PROGRAMAS
df = unionDf(47,48)
print(programas.columns)

['CD_PROGRAMA_IES', 'NM_PROGRAMA_IES']


In [44]:
detalhes_programas = df.select('SG_ENTIDADE_ENSINO', 'NM_ENTIDADE_ENSINO', 
                               'CD_PROGRAMA_IES','AN_INICIO_PROGRAMA', 'DS_SITUACAO_PROGRAMA', 'DT_SITUACAO_PROGRAMA',
                               'NM_PROGRAMA_IDIOMA', 'NM_GRAU_PROGRAMA', 'CD_CONCEITO_PROGRAMA'
                              )
programas = df.select('CD_PROGRAMA_IES', 'NM_PROGRAMA_IES')
programas = programas.dropDuplicates(['CD_PROGRAMA_IES'])
detalhes_programas = detalhes_programas.dropDuplicates()

In [45]:
toParquet(programas, 'capes/capes programas')
toParquet(detalhes_programas, 'capes/capes programas_detalhes')

In [45]:
producao = unionDf(36,46)

In [46]:
producao.columns

['CD_PROGRAMA_IES',
 'NM_PROGRAMA_IES',
 'SG_ENTIDADE_ENSINO',
 'NM_ENTIDADE_ENSINO',
 'AN_BASE',
 'ID_ADD_PRODUCAO_INTELECTUAL',
 'ID_PRODUCAO_INTELECTUAL',
 'NM_PRODUCAO',
 'ID_TIPO_PRODUCAO',
 'NM_TIPO_PRODUCAO',
 'ID_SUBTIPO_PRODUCAO',
 'NM_SUBTIPO_PRODUCAO',
 'ID_FORMULARIO_PRODUCAO',
 'NM_FORMULARIO',
 'ID_AREA_CONCENTRACAO',
 'NM_AREA_CONCENTRACAO',
 'ID_LINHA_PESQUISA',
 'NM_LINHA_PESQUISA',
 'ID_PROJETO',
 'NM_PROJETO',
 'DH_INICIO_AREA_CONC',
 'DH_FIM_AREA_CONC',
 'DH_INICIO_LINHA',
 'DH_FIM_LINHA',
 'IN_GLOSA',
 'IN_PRODUCAO_COM_VINCULO_TCC',
 'ID_ADD_TRABALHO_CONCLUSAO_CT',
 'ID_ADD_FOTO_PROGRAMA',
 'ID_ADD_FOTO_PROGRAMA_IES',
 'CD_IDENTIFICADOR_VEICULO',
 'DS_TITULO_PADRONIZADO']

In [32]:
producao.columns
producao = producao.select(
    'CD_PROGRAMA_IES',
    'NM_PROGRAMA_IES',
    'ID_PRODUCAO_INTELECTUAL',
    'NM_PRODUCAO',
    'NM_TIPO_PRODUCAO',
    'NM_SUBTIPO_PRODUCAO',
    'ID_PROJETO',
    'DS_TITULO_PADRONIZADO',
    'DH_INICIO_AREA_CONC',
    'DH_FIM_AREA_CONC',
    'DH_INICIO_LINHA',
    'DH_FIM_LINHA'
    )
producao = producao.dropDuplicates()

In [33]:
toParquet(producao, name_csvs[36][1])

In [47]:
membros = unionDf(34,35)
membros.columns

['AN_BASE',
 'ID_ADD_FOTO_PROGRAMA',
 'ID_ADD_FOTO_PROGRAMA_IES',
 'AN_INICIO_PROGRAMA',
 'ID_ADD_CONTEXTO',
 'ID_PROJETO',
 'IN_RESPONSAVEL_PROJETO',
 'ID_ADD_MEMBRO_PROJETO',
 'ID_PESSOA',
 'DS_TIPO_MEMBRO',
 'ID_ADD_PARTICIPANTE_EXTERNO',
 'ID_ADD_DISCENTE',
 'ID_ADD_DOCENTE',
 'NM_MEMBRO_PROJETO',
 'NM_PAIS_NACIONALIDADE_MEMBRO',
 'DS_CATEGORIA_MEMBRO_PROJETO',
 'DT_INICIO_VINCULO',
 'DT_FIM_VINCULO',
 'ID_ADD_EGRESSO',
 'ID_ADD_POS_DOC']

In [48]:
print(membros.count())
membros = membros.select('ID_PROJETO', 'ID_PESSOA', 'ID_ADD_DISCENTE', 'ID_ADD_DOCENTE', 
                         'NM_MEMBRO_PROJETO', 'DT_INICIO_VINCULO', 'DT_FIM_VINCULO')
membros = membros.dropDuplicates()
membros.count()

2013159


1772307

In [49]:
toParquet(membros, name_csvs[34][1])

24/10/11 23:22:30 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/10/11 23:22:30 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/10/11 23:22:30 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/10/11 23:22:30 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/10/11 23:22:30 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
24/10/11 23:22:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/10/11 23:22:31 WARN MemoryManager: Total allocation exceeds 95.

In [38]:
financiadores = unionDf(32,33)

In [41]:
financiadores.columns
financiadores = financiadores.select('ID_PROJETO', 'CD_PROGRAMA_IES', 'NM_PROGRAMA_IES', 'NM_FINANCIADOR')
financiadores = financiadores.dropDuplicates()

In [42]:
toParquet(financiadores, name_csvs[32][1])

24/10/11 23:16:14 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/10/11 23:16:14 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/10/11 23:16:14 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/10/11 23:16:14 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [43]:
autores = unionDf(0,12)

In [50]:
autores.columns
autores = autores.select('ID_ADD_PRODUCAO_INTELECTUAL', 'ID_TIPO_PRODUCAO', 'ID_SUBTIPO_PRODUCAO',
                         'ID_PESSOA_DISCENTE', 'ID_PESSOA_DOCENTE', 'NM_AUTOR', 'TP_AUTOR', 'ID_NATUREZA_ATUACAO',
                         'NM_NATUREZA_ATUACAO')
autores = autores.dropDuplicates()

In [51]:
toParquet(autores, name_csvs[0][1])

In [11]:
catalogos = unionDf(13,14)

24/10/13 01:40:03 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [16]:
catalogos = catalogos.select('CD_PROGRAMA', 'NM_PROGRAMA', 'ID_ADD_PRODUCAO_INTELECTUAL', 'ID_PRODUCAO_INTELECTUAL',
                             'NM_SUBTIPO_PRODUCAO', 'NM_AREA_CONCENTRACAO', 'ID_PROJETO', 'DT_TITULACAO', 'DS_PALAVRA_CHAVE',
                             'DS_ABSTRACT', 'DS_KEYWORD', 'IN_TRABALHO_MESMA_AREA', 'ID_PESSOA_DISCENTE', 'ID_PESSOA_ORIENTADOR',
                             'NM_GRANDE_AREA_CONHECIMENTO', 'NM_AREA_CONHECIMENTO', 'NM_SUBAREA_CONHECIMENTO', 'DS_RESUMO')
catalogos = catalogos.dropDuplicates()

In [17]:
toParquet(catalogos, name_csvs[13][1])

In [44]:
discentes = unionDf(28,29)
discentes.columns

['AN_BASE',
 'NM_GRANDE_AREA_CONHECIMENTO',
 'CD_AREA_AVALIACAO',
 'NM_AREA_AVALIACAO',
 'CD_ENTIDADE_CAPES',
 'CD_ENTIDADE_EMEC',
 'SG_ENTIDADE_ENSINO',
 'NM_ENTIDADE_ENSINO',
 'CS_STATUS_JURIDICO',
 'DS_DEPENDENCIA_ADMINISTRATIVA',
 'NM_MODALIDADE_PROGRAMA',
 'NM_GRAU_PROGRAMA',
 'CD_PROGRAMA_IES',
 'NM_PROGRAMA_IES',
 'NM_REGIAO',
 'SG_UF_PROGRAMA',
 'NM_MUNICIPIO_PROGRAMA_IES',
 'CD_CONCEITO_PROGRAMA',
 'CD_CONCEITO_CURSO',
 'ID_PESSOA',
 'TP_DOCUMENTO_DISCENTE',
 'NR_DOCUMENTO_DISCENTE',
 'NM_DISCENTE',
 'NM_PAIS_NACIONALIDADE_DISCENTE',
 'DS_TIPO_NACIONALIDADE_DISCENTE',
 'AN_NASCIMENTO_DISCENTE',
 'DS_FAIXA_ETARIA',
 'DS_GRAU_ACADEMICO_DISCENTE',
 'ST_INGRESSANTE',
 'NM_SITUACAO_DISCENTE',
 'DT_MATRICULA_DISCENTE',
 'DT_SITUACAO_DISCENTE',
 'QT_MES_TITULACAO',
 'NM_TESE_DISSERTACAO',
 'NM_ORIENTADOR_PRINCIPAL',
 'ID_ADD_FOTO_PROGRAMA',
 'ID_ADD_FOTO_PROGRAMA_IES']

In [53]:
discentes.columns
discentes = discentes.select('ID_PESSOA', 'NM_DISCENTE',
                             'TP_DOCUMENTO_DISCENTE', 'NR_DOCUMENTO_DISCENTE',
                             )
discentes = discentes.dropDuplicates(['ID_PESSOA'])
discentes.count()

12560

In [54]:
toParquet(discentes,name_csvs[28][1])

In [47]:
docentes = unionDf(30,31)
docentes.count()

4410

In [51]:
docentes = docentes.select('ID_PESSOA',
                           'NM_DOCENTE', 'TP_DOCUMENTO_DOCENTE', 'NR_DOCUMENTO_DOCENTE')
docentes = docentes.dropDuplicates(['ID_PESSOA'])
docentes.count()

2098

In [52]:
toParquet(docentes,name_csvs[30][1])

In [37]:
cursos = unionDf(15,16)
cursos.columns

['AN_BASE',
 'NM_GRANDE_AREA_CONHECIMENTO',
 'NM_AREA_CONHECIMENTO',
 'NM_SUBAREA_CONHECIMENTO',
 'NM_ESPECIALIDADE',
 'CD_AREA_AVALIACAO',
 'NM_AREA_AVALIACAO',
 'CD_ENTIDADE_CAPES',
 'CD_ENTIDADE_EMEC',
 'SG_ENTIDADE_ENSINO',
 'NM_ENTIDADE_ENSINO',
 'CS_STATUS_JURIDICO',
 'DS_DEPENDENCIA_ADMINISTRATIVA',
 'DS_ORGANIZACAO_ACADEMICA',
 'NM_REGIAO',
 'SG_UF_PROGRAMA',
 'NM_MUNICIPIO_PROGRAMA_IES',
 'CD_PROGRAMA_IES',
 'NM_PROGRAMA_IES',
 'CD_CURSO_PPG',
 'NM_CURSO',
 'NM_GRAU_CURSO',
 'CD_CONCEITO_CURSO',
 'AN_INICIO_PREVISTO',
 'DS_SITUACAO_CURSO',
 'DT_SITUACAO_CURSO',
 'ID_ADD_FOTO_PROGRAMA_IES',
 'ID_ADD_FOTO_PROGRAMA']

In [38]:
print(cursos.count())
cursos = cursos.select('CD_PROGRAMA_IES', 'SG_ENTIDADE_ENSINO', 'NM_ENTIDADE_ENSINO', 'NM_CURSO')
cursos = cursos.dropDuplicates()
cursos.count()

260


96

In [39]:
toParquet(cursos, name_csvs[15][1])

In [10]:
for i in range(17,28):
    df = spark.read.format('csv').option('header', 'true').option('sep', ';')\
        .load(f's3a://{bucket_name}/{csvs[i]}')
    toParquet(df, f"2021-a-2024-detalhes-da-producao-{name_csvs[i][2]}")